<a href="https://colab.research.google.com/github/janaghoniem/Social-Media-Sentiment-Analysis/blob/main/ML_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import vstack
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
import pandas as pd
import numpy as np
import ast
import re
import pickle
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import drive

In [2]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Science/English preprocessing + pipeline & dataset/English_cleaned.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,normalized_tweet,tokens,trigrams_token,target
0,a that's a bummer. you shoulda got david carr ...,"[""'s"", 'bummer', '.', 'shoulda', 'got', 'david...","[('a', 'that', ""'s""), ('that', ""'s"", 'a'), (""'...",-1
1,is upset that he can't update his facebook by ...,"['upset', 'ca', ""n't"", 'updat', 'facebook', 't...","[('is', 'upset', 'that'), ('upset', 'that', 'h...",-1
2,i dived many times for the ball. managed to sa...,"['dive', 'mani', 'time', 'ball', '.', 'manag',...","[('i', 'dive', 'mani'), ('dive', 'mani', 'time...",-1
3,my whole body feels itchy and like its on fire,"['whole', 'bodi', 'feel', 'itchi', 'like', 'fi...","[('my', 'whole', 'bodi'), ('whole', 'bodi', 'f...",-1
4,"no, it's not behaving at all. i'm mad. why am ...","['no', ',', ""'s"", 'not', 'behav', '.', ""'m"", '...","[('no', ',', 'it'), (',', 'it', ""'s""), ('it', ...",-1


1) MLP

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Pre-tokenized list of trigrams
X_tokens = df['trigrams_token'].tolist()
y = df['target']

# Vectorize using CountVectorizer
vectorizer = CountVectorizer(analyzer=lambda x: x)  # x is already tokenized
X = vectorizer.fit_transform(X_tokens)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data: 70% train, 15% validation, 15% test
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y_encoded, test_size=0.15, random_state=42, stratify=y_encoded)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765, random_state=42, stratify=y_train_val)  # ~15% of total

# Train initial MLP model
mlp_model = MLPClassifier(random_state=42, max_iter=300)
mlp_model.fit(X_train, y_train)

# Evaluate initial model
train_acc = accuracy_score(y_train, mlp_model.predict(X_train))
val_acc = accuracy_score(y_val, mlp_model.predict(X_val))
test_acc = accuracy_score(y_test, mlp_model.predict(X_test))

print(f"Initial MLP - Train Accuracy: {train_acc:.4f}")
print(f"Initial MLP - Validation Accuracy: {val_acc:.4f}")
print(f"Initial MLP - Test Accuracy: {test_acc:.4f}")

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001],
    'solver': ['adam'],
    'learning_rate': ['constant', 'adaptive']
}

grid_search = GridSearchCV(MLPClassifier(random_state=42, max_iter=300),
                           param_grid, cv=3, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# Evaluate tuned model
train_acc_tuned = accuracy_score(y_train, best_model.predict(X_train))
val_acc_tuned = accuracy_score(y_val, best_model.predict(X_val))
test_acc_tuned = accuracy_score(y_test, best_model.predict(X_test))

print("\nBest Hyperparameters:", grid_search.best_params_)
print(f"Tuned MLP - Train Accuracy: {train_acc_tuned:.4f}")
print(f"Tuned MLP - Validation Accuracy: {val_acc_tuned:.4f}")
print(f"Tuned MLP - Test Accuracy: {test_acc_tuned:.4f}")


Initial MLP - Train Accuracy: 0.6301
Initial MLP - Validation Accuracy: 0.6254
Initial MLP - Test Accuracy: 0.6235
Fitting 3 folds for each of 24 candidates, totalling 72 fits


### 1) Random Forest

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Tokenized trigrams and labels
X_tokens = df['trigrams_token'].tolist()  # Pre-tokenized trigrams
y = df['target']

# Vectorization
vectorizer = CountVectorizer(analyzer=lambda x: x)  # Accepts tokenized input
X = vectorizer.fit_transform(X_tokens)

# Label encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train/validation/test split (70/15/15)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_encoded, test_size=0.15, random_state=42, stratify=y_encoded)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42, stratify=y_train_val)  # 0.1765 * 0.85 ≈ 0.15

# Train initial Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate initial model
train_acc = accuracy_score(y_train, rf_model.predict(X_train))
val_acc = accuracy_score(y_val, rf_model.predict(X_val))
test_acc = accuracy_score(y_test, rf_model.predict(X_test))

print(f"Initial RF - Train Accuracy: {train_acc:.4f}")
print(f"Initial RF - Validation Accuracy: {val_acc:.4f}")
print(f"Initial RF - Test Accuracy: {test_acc:.4f}")

# Hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

best_rf_model = grid_search.best_estimator_

# Evaluate tuned model
train_acc_tuned = accuracy_score(y_train, best_rf_model.predict(X_train))
val_acc_tuned = accuracy_score(y_val, best_rf_model.predict(X_val))
test_acc_tuned = accuracy_score(y_test, best_rf_model.predict(X_test))

print("\nBest Hyperparameters:", grid_search.best_params_)
print(f"Tuned RF - Train Accuracy: {train_acc_tuned:.4f}")
print(f"Tuned RF - Validation Accuracy: {val_acc_tuned:.4f}")
print(f"Tuned RF - Test Accuracy: {test_acc_tuned:.4f}")


### 2) SVM

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.preprocessing import LabelEncoder
# from sklearn.svm import SVC
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.metrics import accuracy_score

# # Your existing code
# X_tokens = df['trigrams_token'].tolist()  # Pre-tokenized list of trigrams
# y = df['target']

# # Vectorize using CountVectorizer
# vectorizer = CountVectorizer(analyzer=lambda x: x)  # Pre-tokenized input
# X = vectorizer.fit_transform(X_tokens)

# # Encode labels
# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(y)

# # Split data: train 70%, validation 15%, test 15%
# X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_encoded, test_size=0.15, random_state=42, stratify=y_encoded)
# X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42, stratify=y_train_val)  # ~15% of total

# # Train initial SVM model
# svm_model = SVC(kernel='linear', random_state=42)
# svm_model.fit(X_train, y_train)

# # Evaluate initial model
# train_acc = accuracy_score(y_train, svm_model.predict(X_train))
# val_acc = accuracy_score(y_val, svm_model.predict(X_val))
# test_acc = accuracy_score(y_test, svm_model.predict(X_test))

# print(f"Initial SVM - Train Accuracy: {train_acc:.4f}")
# print(f"Initial SVM - Validation Accuracy: {val_acc:.4f}")
# print(f"Initial SVM - Test Accuracy: {test_acc:.4f}")

# # Hyperparameter tuning with GridSearchCV
# param_grid = {
#     'C': [0.1, 1, 10],
#     'kernel': ['linear', 'rbf'],
#     'gamma': ['scale', 'auto']
# }

# grid_search = GridSearchCV(SVC(random_state=42), param_grid, cv=3, scoring='accuracy', verbose=1)
# grid_search.fit(X_train, y_train)

# best_model = grid_search.best_estimator_

# # Evaluate tuned model
# train_acc_tuned = accuracy_score(y_train, best_model.predict(X_train))
# val_acc_tuned = accuracy_score(y_val, best_model.predict(X_val))
# test_acc_tuned = accuracy_score(y_test, best_model.predict(X_test))

# print("\nBest Hyperparameters:", grid_search.best_params_)
# print(f"Tuned SVM - Train Accuracy: {train_acc_tuned:.4f}")
# print(f"Tuned SVM - Validation Accuracy: {val_acc_tuned:.4f}")
# print(f"Tuned SVM - Test Accuracy: {test_acc_tuned:.4f}")


### 3) Logistic regression

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

# Keep the trigram list as is
X_tokens = df['trigrams_token'].tolist()  # List of lists of trigram strings
y = df['target']

# Vectorize using CountVectorizer with a custom analyzer that expects pre-tokenized input
vectorizer = CountVectorizer(analyzer=lambda x: x)
X = vectorizer.fit_transform(X_tokens)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into train (60%) and temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_encoded, test_size=0.4, stratify=y_encoded, random_state=42
)

# Split temp into validation (20%) and test (20%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# Train baseline Logistic Regression model
logreg_model = LogisticRegression(max_iter=1000, random_state=42)
logreg_model.fit(X_train, y_train)

# Predictions before tuning
y_train_pred = logreg_model.predict(X_train)
y_val_pred = logreg_model.predict(X_val)
y_test_pred = logreg_model.predict(X_test)

print(f"Baseline Logistic Regression Train Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
print(f"Baseline Logistic Regression Validation Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Baseline Logistic Regression Test Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")

print("\n--- Baseline Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_.astype(str)))

# -------------------- Hyperparameter Tuning --------------------
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],      # Inverse of regularization strength
    'penalty': ['l2'],                # L2 penalty (default and preferred)
    'solver': ['lbfgs'],              # Good solver for multiclass problems
}

grid_search = GridSearchCV(
    LogisticRegression(max_iter=1000, random_state=42),
    param_grid,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit grid search only on training data
grid_search.fit(X_train, y_train)
best_logreg = grid_search.best_estimator_

print("\nBest hyperparameters:", grid_search.best_params_)

# Predictions after tuning
y_train_pred_tuned = best_logreg.predict(X_train)
y_val_pred_tuned = best_logreg.predict(X_val)
y_test_pred_tuned = best_logreg.predict(X_test)

print(f"\nTuned Logistic Regression Train Accuracy: {accuracy_score(y_train, y_train_pred_tuned):.4f}")
print(f"Tuned Logistic Regression Validation Accuracy: {accuracy_score(y_val, y_val_pred_tuned):.4f}")
print(f"Tuned Logistic Regression Test Accuracy: {accuracy_score(y_test, y_test_pred_tuned):.4f}")

print("\n--- Tuned Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred_tuned, target_names=label_encoder.classes_.astype(str)))


Baseline Logistic Regression Train Accuracy: 0.5984
Baseline Logistic Regression Validation Accuracy: 0.5986
Baseline Logistic Regression Test Accuracy: 0.5975

--- Baseline Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.60      0.60      0.60    141567
           1       0.59      0.60      0.59    138451

    accuracy                           0.60    280018
   macro avg       0.60      0.60      0.60    280018
weighted avg       0.60      0.60      0.60    280018

Fitting 3 folds for each of 5 candidates, totalling 15 fits

Best hyperparameters: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}

Tuned Logistic Regression Train Accuracy: 0.5984
Tuned Logistic Regression Validation Accuracy: 0.5986
Tuned Logistic Regression Test Accuracy: 0.5975

--- Tuned Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.60      0.60      0.60    141567
           1       0.59  

### 4) Naive Bayes

In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

# Keep the trigram list as is
X_tokens = df['trigrams_token'].tolist()  # List of lists of trigram strings
y = df['target']

# Vectorize using CountVectorizer with a custom analyzer that expects pre-tokenized input
vectorizer = CountVectorizer(analyzer=lambda x: x)
X = vectorizer.fit_transform(X_tokens)

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into train (60%) and temp (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_encoded, test_size=0.4, stratify=y_encoded, random_state=42
)

# Split temp into validation (20%) and test (20%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# Train baseline Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Predictions before tuning
y_train_pred = nb_model.predict(X_train)
y_val_pred = nb_model.predict(X_val)
y_test_pred = nb_model.predict(X_test)

print(f"Baseline NB Train Accuracy: {accuracy_score(y_train, y_train_pred):.4f}")
print(f"Baseline NB Validation Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Baseline NB Test Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")

print("\n--- Baseline Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_.astype(str)))

# -------------------- Hyperparameter Tuning --------------------
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0]  # Laplace smoothing parameter
}

grid_search = GridSearchCV(
    MultinomialNB(),
    param_grid,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit grid search only on training data
grid_search.fit(X_train, y_train)
best_nb = grid_search.best_estimator_

print("\nBest hyperparameters:", grid_search.best_params_)

# Predictions after tuning
y_train_pred_tuned = best_nb.predict(X_train)
y_val_pred_tuned = best_nb.predict(X_val)
y_test_pred_tuned = best_nb.predict(X_test)

print(f"\nTuned NB Train Accuracy: {accuracy_score(y_train, y_train_pred_tuned):.4f}")
print(f"Tuned NB Validation Accuracy: {accuracy_score(y_val, y_val_pred_tuned):.4f}")
print(f"Tuned NB Test Accuracy: {accuracy_score(y_test, y_test_pred_tuned):.4f}")

print("\n--- Tuned Classification Report (Test Set) ---")
print(classification_report(y_test, y_test_pred_tuned, target_names=label_encoder.classes_.astype(str)))


Baseline NB Train Accuracy: 0.5817
Baseline NB Validation Accuracy: 0.5832
Baseline NB Test Accuracy: 0.5811

--- Baseline Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.58      0.60      0.59    141567
           1       0.58      0.57      0.57    138451

    accuracy                           0.58    280018
   macro avg       0.58      0.58      0.58    280018
weighted avg       0.58      0.58      0.58    280018

Fitting 3 folds for each of 5 candidates, totalling 15 fits

Best hyperparameters: {'alpha': 10.0}

Tuned NB Train Accuracy: 0.5817
Tuned NB Validation Accuracy: 0.5832
Tuned NB Test Accuracy: 0.5811

--- Tuned Classification Report (Test Set) ---
              precision    recall  f1-score   support

          -1       0.58      0.60      0.59    141567
           1       0.58      0.57      0.57    138451

    accuracy                           0.58    280018
   macro avg       0.58      0.58      0.58    